In [1]:
from types import SimpleNamespace
import torch
from dataset import get_mnist
from networks import CheckerboardCouplingLayer

In [2]:
args = SimpleNamespace()
args.input_dim = 784   # 28x28
args.hidden_dim = 256
args.num_layer = 5
args.lr = 1e-3
args.batch = 64
args.epoch = 10
print(args)

namespace(input_dim=784, hidden_dim=256, num_layer=5, lr=0.001, batch=64, epoch=10)


In [3]:
loader = get_mnist(args.batch)
len(loader.train), len(loader.test)

(937, 157)

In [4]:
couplelayer = CheckerboardCouplingLayer(args.input_dim, args.hidden_dim)
couplelayer

CheckerboardCouplingLayer(
  (scale_net): Sequential(
    (0): Linear(in_features=392, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=392, bias=True)
    (3): Tanh()
  )
  (translate_net): Sequential(
    (0): Linear(in_features=392, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=392, bias=True)
  )
)

In [5]:
x = torch.randn(10, 784)
z, _ = couplelayer(x)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x784 and 392x256)